# Análise exploratória de dados

In [1]:
import pandas as pd
import seaborn as sn

## 0. Coleta de Dados

In [2]:
fmri = sn.load_dataset('fmri')
fmri.to_csv('../data/raw/fmriData.csv', index = False)

## 1. Dicionário de dados

In [3]:
fmri.columns

Index(['subject', 'timepoint', 'event', 'region', 'signal'], dtype='object')

In [4]:
fmri_dict = pd.DataFrame([
    {
        'variavel': 'subject',
        'descricao': 'A pessoa em que foi realizado o exame',
        'tipo': 'qualitativa',
        'subtipo': 'nominal', 
    },{
        'variavel': 'timepoint',
        'descricao': 'Período no tempo em que foi feito o exame',
        'tipo': 'qualitativa',
        'subtipo': 'ordinal',
    },{
        'variavel': 'event',
        'descricao': 'Forma como o cérebro foi estimulado para coleta de dados',
        'tipo': 'qualitativa',
        'subtipo': 'nominal',
      },{
        'variavel': 'region',
        'descricao': 'Região do cérebro examinada',
        'tipo': 'qualitativa',
        'subtipo': 'nominal',
    },{
        'variavel': 'signal',
        'descricao': 'Representa a atividade cerebral na região determinada',
        'tipo': 'quantitativa',
        'subtipo': 'contínua',
    }
])

In [5]:
fmri_dict

,variavel,descricao,tipo,subtipo
0,subject,A pessoa em que foi realizado o exame,qualitativa,nominal
1,timepoint,Período no tempo em que foi feito o exame,qualitativa,ordinal
2,event,Forma como o cérebro foi estimulado para colet...,qualitativa,nominal
3,region,Região do cérebro examinada,qualitativa,nominal
4,signal,Representa a atividade cerebral na região dete...,quantitativa,contínua


In [6]:
fmri_dict.to_csv('../data/external/fmri_dict.csv', index=False)